In [16]:
# 파이썬데이터분석 중간 발표
# 202104309 이진용 / 202104247 유민우 / 202104389 황형진
# 주제 : 2017년도 서울특별시 기상별 사고율 데이터 분석
# 데이터 출처 
# 기상 데이터 : https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
# 교통사고 데이터 : https://www.data.go.kr/data/15094191/fileData.do?recommendDataYn=Y

In [24]:
import pandas as pd

# 서울시 2017년 일자별 날씨 현황 데이터 불러오기 
df = pd.read_csv('/Users/hwanghyeongjin/Desktop/PDA_ver1.0/서울특별시 일별 기상.csv', encoding='cp949')

# 일시를 datatime 형식으로 변환
df['일시'] = pd.to_datetime(df['일시'], format='%Y.%m.%d %H:%M')


df['날짜'] = df['일시'].dt.date
df['강수량(mm)'] = pd.to_numeric(df['강수량(mm)'], errors='coerce').fillna(0)

# groupby로 날짜와 강수량 묶기
df_daily = df.groupby('날짜')['강수량(mm)'].sum().reset_index()

print(df_daily)

             날짜  강수량(mm)
0    2017-01-01      0.0
1    2017-01-02      0.3
2    2017-01-03      0.0
3    2017-01-04      0.0
4    2017-01-05      0.0
..          ...      ...
360  2017-12-27      0.0
361  2017-12-28      0.0
362  2017-12-29      0.0
363  2017-12-30      0.5
364  2017-12-31      0.2

[365 rows x 2 columns]


In [26]:
import pandas as pd

# 교통사고 데이터 불러오기
df_accident = pd.read_excel('/Users/hwanghyeongjin/Desktop/PDA_ver1.0/seoul_car_accident.xlsx')

#'일자'를 datetime 형식으로 변환
df_accident['일자'] = pd.to_datetime(df_accident['일자'], format='%Y%m%d')

# 날씨 데이터 준비
df_daily['날짜'] = pd.to_datetime(df_daily['날짜'], format='%Y-%m-%d')
df_weather = df_daily.rename(columns={'날짜': '일자'})

# 데이터 병합 (df_weather)
df_merged = pd.merge(df_accident, df_weather, on='일자', how='inner')

# 결과 출력
print(df_merged.head())

          일자  서울발생건수  서울사망자수  서울부상자수  강수량(mm)
0 2017-01-01      73       2     125      0.0
1 2017-01-02      82       4     136      0.3
2 2017-01-03      88       1     118      0.0
3 2017-01-04     115       1     163      0.0
4 2017-01-05     103       1     153      0.0


In [28]:
# 1. '비가 온 날'과 '안 온 날' 구분
# 강수량(mm)이 0보다 크면 비 온 날
df_merged['비옴여부'] = df_merged['강수량(mm)'].apply(lambda x: '비옴' if x > 0 else '비안옴')

# 2. 그룹별 평균 계산
grouped = df_merged.groupby('비옴여부')[['서울발생건수', '서울사망자수', '서울부상자수']].mean()

# 3. 결과 출력
print(grouped)

          서울발생건수    서울사망자수      서울부상자수
비옴여부                                  
비안옴   106.946768  0.977186  148.490494
비옴    102.921569  0.843137  144.676471


In [30]:
# 1. 강수량 구간 설정
bins = [-0.1, 0, 5, 20, float('inf')]  # (주의: 0mm를 따로 구분하기 위해 -0.1 시작)
labels = ['0mm (맑음)', '0.1~5mm (약한 비)', '5.1~20mm (보통 비)', '20mm 이상 (강한 비)']

df_merged['강수량구간'] = pd.cut(df_merged['강수량(mm)'], bins=bins, labels=labels)

# 2. 구간별 평균 사고 통계 계산
grouped_by_rain = df_merged.groupby('강수량구간')[['서울발생건수', '서울사망자수', '서울부상자수']].mean()

# 3. 결과 출력
print(grouped_by_rain)

                     서울발생건수    서울사망자수      서울부상자수
강수량구간                                            
0mm (맑음)         106.946768  0.977186  148.490494
0.1~5mm (약한 비)   100.691176  0.852941  142.500000
5.1~20mm (보통 비)  106.444444  0.666667  146.166667
20mm 이상 (강한 비)   108.437500  1.000000  152.250000


/var/folders/2l/qbkgdpjj0gdfj5sl82smp9ww0000gn/T/ipykernel_35258/2460916080.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_rain = df_merged.groupby('강수량구간')[['서울발생건수', '서울사망자수', '서울부상자수']].mean()
